### Setup

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../')
from bert_mpi import *

### Toy Example

In [4]:
from bert_mpi import *
from transformers import AutoTokenizer, BertForMultipleChoice, BertLMHeadModel
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = BertForMultipleChoice.from_pretrained("bert-base-uncased")
model = BertLMHeadModel.from_pretrained("bert-base-uncased")

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [37]:
t1 = "Make friends easily"
t2 = "Trust other people"
t3 = "Have difficulty imagining things"
t_lst = [t1, t2, t3]
prompt_lst = [prepare_mpi_questions(t) for t in t_lst]
# ic(prompt_lst[0])
inputs = tokenizer(prompt_lst[0] +  "A", return_tensors="pt")
# outputs = model(**inputs, labels=inputs["input_ids"])
outputs = model(**inputs)
loss = outputs.loss
logits = outputs.logits

ic| prob.shape: torch.Size([67, 30522])
ic| torch.sum(torch.log(logits)): tensor(-6.6448, grad_fn=<SumBackward0>)
ic| logits: tensor([0.0278, 0.2255, 0.9994, 0.9981, 0.9998, 0.9954, 0.9995, 1.0000, 0.9994,
                    0.9979, 0.9877, 0.9978, 1.0000, 1.0000, 0.9717, 0.8981, 0.8578, 0.9991,
                    0.9992, 0.9904, 0.9999, 0.8551, 1.0000, 0.9916, 0.9933, 0.9986, 0.9736,
                    0.9981, 1.0000, 0.9947, 1.0000, 0.9998, 0.9999, 1.0000, 1.0000, 0.9971,
                    0.9782, 1.0000, 0.9990, 1.0000, 0.9999, 0.9404, 0.9561, 1.0000, 0.9912,
                    0.9978, 1.0000, 0.9983, 0.9956, 0.9996, 0.6608, 1.0000, 0.9995, 1.0000,
                    1.0000, 0.9520, 0.7708, 1.0000, 0.9990, 0.9816, 1.0000, 0.9985, 0.9171,
                    0.9871, 1.0000, 0.9685, 0.9895], grad_fn=<MaxBackward0>)
ic| logits.shape: torch.Size([67])
ic| loss: None


In [ ]:
# prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
# choice0 = "It is eaten with a fork and a knife."
# choice1 = "It is eaten while held in the hand."
# labels = torch.tensor(0).unsqueeze(0)  # choice0 is correct (according to Wikipedia ;)), batch size 1
# statement = "Trust others."
t1 = "Make friends easily"
t2 = "Trust other people"
t3 = "Have difficulty imagining things"
t_lst = [t1, t2, t3]
prompt_lst = [prepare_mpi_questions(t) for t in t_lst]


# prompt_lst = []
for prompt in prompt_lst:
    encoding = tokenizer([prompt]*5, ['A', 'B', 'C', 'D', 'E'], return_tensors="pt", padding=True)
    outputs = model(**{k: v.unsqueeze(0) for k, v in encoding.items()})  # batch size is 1

    loss = outputs.loss
    logits = outputs.logits
    ic(logits)
    print(MPI_IDX_TO_KEY[torch.argmax(logits.squeeze()).item()])

### MPI Experiment

In [7]:
from bert_mpi import *
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# tokenizer = AutoTokenizer.from_pretrained("bert-large-cased")
# model = BertForMultipleChoice.from_pretrained("bert-base-uncased")
# model = BertForMultipleChoice.from_pretrained("bert-large-cased")

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = BertForMultipleChoice.from_pretrained("bert-base-uncased")
model = BertLMHeadModel.from_pretrained("bert-base-uncased")
filename = "mpi_small"
local_path = "../Dataset/" + f"{filename}.csv"

MPI_OBJ_LST = []
idx_lst = [(0, 120)]
for (start, end) in idx_lst:
    mpi = MPI(local_path, start, end)
    mpi.run(tokenizer, model)
    mpi.display_score()
    MPI_OBJ_LST.append(mpi)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
ic| self.questions.shape: (120,)
ic| Counter(col): Counter({'N': 24, 'E': 24, 'O': 24, 'A': 24, 'C': 24})
ic| len(Counter(col)): 5


There are 120 multiple choice questions in total.


ic| Counter(col): Counter({1: 65, -1: 55})
ic| len(Counter(col)): 2
  0%|          | 0/120 [00:00<?, ?it/s]

(A) Very Accurate: -0.05559594929218292
(B) Moderately Accurate: -0.08758116513490677
(C) Neither Accurate Nor Inaccurate: -0.08514217287302017
(D) Moderately Inaccurate: -0.08333847671747208


ic| f"ANSWER: {MPI_IDX_TO_KEY[pred.item()]}": 'ANSWER: A'
  1%|          | 1/120 [00:01<03:50,  1.93s/it]

...

ic| f"ANSWER: {MPI_IDX_TO_KEY[pred.item()]}": 'ANSWER: A'
 16%|█▌        | 19/120 [00:30<02:52,  1.71s/it]

(E) Very Inaccurate: -0.04411669075489044
(A) Very Accurate: -0.054008714854717255


 16%|█▌        | 19/120 [00:31<02:45,  1.64s/it]

(B) Moderately Accurate: -0.06719972938299179


KeyboardInterrupt: 